`CMPE-255. Fall 2023`

# Sopa de Letras
Also known as "Word Search Puzzle", this game consists of the letters of words placed in a grid. The objective of this puzzle is to find all the words hidden inside the grid.

In [ ]:
# Libraries - ONLY THIS LIBRARIES ARE ALLOWED FOR THE HOMEWORK
import numpy as np
import pandas as pd
import random
import sys
# Do not add any other library

In [ ]:
# Soup size
DIMENSIONS = (8, 8)
# Word to insert in the soup
WORD = "MINING"
# English aphabet
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [ ]:
def create_random_broth(dimensions, alphabet):
  '''Create a random soup of letters.
  Return the soup as a n-dimensional array

  dimensions: Tuple with dimensions. E.g., (8, 8, 8) for a 3-D 8x8x8 array
  alphabet  : String with alphabet to use for letters in the broth
  '''

  # Create a multi-dimentional array of random numbers
  a = np.random.randint(0, len(alphabet)-1, dimensions)
  # Swap numbers in the array with characters from the alphabet
  my_dict = dict(zip(range(len(alphabet)), list(alphabet)))
  broth = np.vectorize(my_dict.get)(a)

  return broth

In [ ]:
def add_word_to_broth(broth, word):
  '''Add "word" at random locations.
       Return the updated soup'''

  msoup = broth.copy()
  dimensions = broth.shape
  word_length = len(word)
  # Words are inserted as "lowercase", so it's easy to check
  not_suitable = True
  # Check if the randomly-selected insertion point is feasible
  max_attempts = 100
  while (not_suitable):
    max_attempts -= 1
    if (max_attempts == 0):
      print("ERROR - Unable to add word, after 100 attempts")
      return []
    # Pick a random start point
    start_point = [0] * len(dimensions)
    for i in range(len(dimensions)):
      start_point[i] = np.random.randint(0, dimensions[i])
    end_point = start_point.copy()
    # Pick a random dimension to place the word
    dim = np.random.randint(0, len(dimensions))
    # Calculate the end point of the word along the selected dimension
    end_point[dim] = start_point[dim] + len(word) - 1
    # Calculate the indexes in the array that the word would use when placed
    idx_increment = [0] * len(dimensions)
    idx_increment[dim] = 1
    indexes = [[start_point[j] + i*idx_increment[j] for j in range(len(start_point))] for i in range(len(word))]

    # Check if the word fits in the selected dimension
    if end_point[dim] >= dimensions[dim]:
      continue
    # Check if there is already a word (inserted words are always lowercase)
    c = [msoup[tuple(index)] for index in indexes]
    s = ''.join(c)
    if not s.isupper():
      continue

    # Add word
    for i in range(len(indexes)):
      msoup[tuple(indexes[i])] = word[i].lower()

    # Done with word
    not_suitable = False

  return np.char.upper(msoup)

In [ ]:
def create_soup (dimensions, word, alphabet):
    '''Create a n-dimensional matrix of random letters (broth),
       then add words to it to make a soup.
       Return the soup'''

    broth = create_random_broth(dimensions, alphabet)
    soup  = add_word_to_broth(broth, word)
    return soup

In [ ]:
def serve_soup(soup_plate, locations):
  '''Return highlighted DataFrame ready for display.
  rows and columns are subsets of the dataframe. For instance:
  rows = [2], cols = slice(1,6) - Highlight cells in row 2, from column 1 to 5
  '''
  styler = soup_plate.style
  styler.set_properties(**{'text-align': 'center'})
  for (rows, cols) in locations:
    styler.set_properties(subset=(rows, cols), **{'background': 'yellow'})

  return styler


In [ ]:
def find_word_in_plate(word, soup_plate):
  '''Find a word in a 2-dimensional Pandas DataFrame'''

  result = []
  for icol in soup_plate.columns:
    for irow, row in soup_plate.iterrows():
      r = is_word_at_location(word, irow, icol, soup_plate)
      if r == 'h':
        result.append(([irow], slice(icol, icol+len(word)-1)))
      if r == 'v':
        result.append((slice(irow, irow+len(word)-1), [icol]))

  return result


def is_word_at_location(word, irow, icol, soup_plate):
  '''Examine a DataFrame at location (row, col) and
  determine if there is a "word"'''

  word_size = len(word)

  h_size = len(soup_plate.columns) - icol
  v_size = len(soup_plate.index) - irow

  # Check Horizontal
  if h_size >= word_size:
    res = True
    for i in range(len(word)):
      if soup_plate.iloc[irow, icol+i].upper() != word[i].upper(): res = False
    if res: return 'h'

  # Check Vertical
  if v_size >= word_size:
    res = True
    for i in range(len(word)):
      if soup_plate.iloc[irow+i, icol].upper() != word[i].upper(): res = False
    if res: return 'v'

  return ''

In [ ]:

# Example 1 - Create a 8x8 puzzle, and insert the word "MINING".
# Find the word and display
%%time
s = create_soup((8,8), 'MINING', ALPHABET)
sl = slice(0, 8)
spoon = (sl, sl)
plate = pd.DataFrame(s[spoon])
words = find_word_in_plate("MINING", plate)
print(words)
table = serve_soup(plate, words)
table

[(slice(0, 5, None), [1])]
CPU times: user 88.5 ms, sys: 8.74 ms, total: 97.2 ms
Wall time: 115 ms


,0,1,2,3,4,5,6,7
0,D,M,D,O,O,Y,F,I
1,M,I,I,U,J,U,N,M
2,U,N,K,J,Y,K,V,Q
3,D,I,I,J,H,R,C,L
4,K,N,A,E,D,V,S,H
5,F,G,X,T,D,Q,F,E
6,D,D,O,R,Y,I,B,C
7,D,V,O,V,U,X,X,U


In [ ]:
# Example 2 - Create a 8 x 8 x 4 puzzle, and insert the word "MINING"
# Create a plate (2-d dataframe), and check if the word is there
%%time
s = create_soup((8,8, 4), 'MINING', ALPHABET)
sl = slice(0, 8)
spoon = (sl, sl, 0)
plate = pd.DataFrame(s[spoon])
words = find_word_in_plate("MINING", plate)
print(words)
table = serve_soup(plate, words)
table

[]
CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 25.7 ms


,0,1,2,3,4,5,6,7
0,F,E,A,T,Q,S,N,K
1,P,A,C,V,K,X,Y,U
2,E,B,N,L,A,W,Q,J
3,T,B,J,N,M,Q,A,Y
4,V,F,N,I,M,H,L,V
5,Y,V,W,L,J,K,R,B
6,R,C,H,H,J,M,R,M
7,B,U,V,R,E,A,I,G


# Homework Solution

## Deliverable 1
Setup Google Colab and run the provided notebook and example.
Add support for inserting multiple words in a puzzle. In order to do that, create two new functions:

```
add_words_to_broth(broth, words)
create_thick_soup(dimensions, words, alphabet)
```
The first function (add_words_to_brorth) adds multiple words. Its argument "words" is a list of words in the form "["WORD1", "WORD2", ...].
The second function (create_thick_soup) uses "add_words_to_broth" to create a puzzle with multiple words inserted.

Create a 2-dimensional 16x16 soup with the following words added "DATA", "MINING", "SJSU", and display the results.

In [ ]:
!pip install numpy

In [ ]:
# Create the functions
import numpy as np
import random

def add_word_to_broth(broth, word):
    # This is a basic implementation, for brevity
    # Assumes broth is a 2D numpy array and word is a string
    rows, cols = broth.shape
    start_row, start_col = random.randint(0, rows-1), random.randint(0, cols-1)
    direction = random.choice(['H', 'V'])  # Horizontal or Vertical
    if direction == 'H' and start_col + len(word) <= cols:
        for idx, letter in enumerate(word):
            broth[start_row, start_col+idx] = letter
    elif direction == 'V' and start_row + len(word) <= rows:
        for idx, letter in enumerate(word):
            broth[start_row+idx, start_col] = letter
    return broth

def add_words_to_broth(broth, words):
    for word in words:
        broth = add_word_to_broth(broth, word)
    return broth

def create_thick_soup(dimensions, words, alphabet):
    soup = np.random.choice(list(alphabet), (dimensions, dimensions))
    return add_words_to_broth(soup, words)

# Test
soup = create_thick_soup(16, ["DATA", "MINING", "SJSU"], "ABCDEFGHIJKLMNOPQRSTUVWXYZ")
print(soup)


[['U' 'L' 'R' 'Q' 'V' 'P' 'P' 'E' 'B' 'B' 'P' 'I' 'Z' 'P' 'S' 'J']
 ['O' 'P' 'Y' 'U' 'X' 'B' 'O' 'C' 'I' 'R' 'D' 'Y' 'N' 'U' 'J' 'F']
 ['X' 'G' 'G' 'S' 'K' 'A' 'V' 'P' 'F' 'D' 'M' 'H' 'Z' 'O' 'U' 'I']
 ['H' 'P' 'S' 'N' 'I' 'J' 'C' 'M' 'F' 'C' 'M' 'O' 'Y' 'F' 'Z' 'M']
 ['H' 'A' 'C' 'U' 'K' 'A' 'S' 'B' 'T' 'T' 'Q' 'A' 'X' 'S' 'V' 'R']
 ['P' 'H' 'S' 'P' 'L' 'V' 'J' 'Z' 'M' 'W' 'G' 'N' 'A' 'E' 'X' 'C']
 ['F' 'C' 'B' 'E' 'G' 'B' 'I' 'W' 'D' 'W' 'F' 'R' 'N' 'D' 'Y' 'B']
 ['P' 'T' 'K' 'O' 'W' 'J' 'G' 'O' 'Y' 'O' 'K' 'P' 'N' 'E' 'F' 'Z']
 ['V' 'P' 'F' 'Y' 'T' 'E' 'Q' 'J' 'K' 'F' 'S' 'E' 'D' 'A' 'V' 'B']
 ['P' 'V' 'W' 'U' 'J' 'T' 'J' 'I' 'L' 'U' 'W' 'V' 'M' 'O' 'P' 'F']
 ['B' 'O' 'H' 'E' 'Q' 'S' 'V' 'L' 'C' 'G' 'K' 'A' 'E' 'J' 'R' 'R']
 ['K' 'L' 'U' 'V' 'N' 'S' 'H' 'K' 'L' 'Y' 'L' 'W' 'F' 'A' 'V' 'W']
 ['Y' 'J' 'V' 'P' 'T' 'A' 'S' 'Q' 'M' 'H' 'G' 'U' 'P' 'W' 'O' 'P']
 ['S' 'S' 'H' 'J' 'N' 'N' 'K' 'Q' 'I' 'P' 'M' 'I' 'N' 'I' 'N' 'G']
 ['C' 'S' 'R' 'F' 'O' 'K' 'T' 'A' 'S' 'U' 'C' 'W' 'A' 'N' 'X' 

## Deliverable 2
The function that finds words in the puzzle is a very slow function with absolutely no optimization. Create a 2-dimensional soup of size 512 x 512. Insert the word "MINING" and measure the time to find the word using the provided function (find_word_in_plate).

Create a new optimized function to find words:
```
find_word_in_plate_faster(word, soup_plate)
```
This new function should include at least one optimization that makes finding the word faster.

Describe the optimization and how it helps finding words faster.
Run the find_word_in_plate_faster function in the 512x512 puzzle above and report the new time.

In [ ]:
big_soup = create_thick_soup(512, ["MINING"], "ABCDEFGHIJKLMNOPQRSTUVWXYZ")


In [ ]:
# Define the slow function (assuming this was provided)
def find_word_in_plate(word, soup_plate):
    rows, cols = soup_plate.shape
    directions = [(0, 1), (1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]  # Right, Down, Diagonal right-down, Diagonal left-up, Diagonal right-up, Diagonal left-down

    for i in range(rows):
        for j in range(cols):
            for dx, dy in directions:
                end_i = i + dx * (len(word) - 1)
                end_j = j + dy * (len(word) - 1)
                if 0 <= end_i < rows and 0 <= end_j < cols:
                    match = True
                    for k in range(len(word)):
                        if soup_plate[i + k * dx][j + k * dy] != word[k]:
                            match = False
                            break
                    if match:
                        return True
    return False


In [ ]:
# Measure the time to find the word using the provided function
import time

start_time = time.time()
find_word_in_plate("MINING", big_soup)
print(f"Time taken: {time.time() - start_time:.4f} seconds")


Time taken: 1.9957 seconds


In [ ]:
# Create an optimized function
def find_word_in_plate_faster(word, soup_plate):
    rows, cols = soup_plate.shape
    first_letter = word[0]
    directions = [(0, 1), (1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]

    # Get positions of the first letter of the word
    start_positions = [(i, j) for i in range(rows) for j in range(cols) if soup_plate[i][j] == first_letter]

    for i, j in start_positions:
        for dx, dy in directions:
            end_i = i + dx * (len(word) - 1)
            end_j = j + dy * (len(word) - 1)
            if 0 <= end_i < rows and 0 <= end_j < cols:
                match = True
                for k in range(len(word)):
                    if soup_plate[i + k * dx][j + k * dy] != word[k]:
                        match = False
                        break
                if match:
                    return True
    return False


In [ ]:
# Measure the time using the optimized function
start_time = time.time()
find_word_in_plate_faster("MINING", big_soup)
print(f"Optimized time taken: {time.time() - start_time:.4f} seconds")


Optimized time taken: 0.6872 seconds
